# 1.0 Extract movement and track positions over time.

For each video we use YOLOv8 to extract movement data as a set of body keypoints and use its `model.track` method to track individuals over time.


# 1.1 Video pose estimation with Ultralytics YOLO
[Ultralytics](https://github.com/ultralytics/ultralytics) is a commercially maintained version of the YOLO object recognition model. [Yolov7](https://github.com/WongKinYiu/yolov7) introduced pose estimation and v8 improves the models and makes everything much more user-friendly. The current version is YOLOv11. It can be installed as a package

* Pip : `pip install ultralytics`
* Conda : `conda install -c conda-forge ultralytics`

## 1.2 Object tracking 

Since YOLOv8, it also comes with a `model.track` method. This aims to keep track of all identified objects over the course of a video. Let's make use of that to track individuals over time. 

This is pretty easy instead of calling 
`results = model(video_path, stream=True)`

we can call
`results = model.track(video_path, stream=True)`

https://docs.ultralytics.com/modes/track/#persisting-tracks-loop

In [ ]:
import os
import sys

project_root = os.path.join("..")
sys.path.append(project_root)

# Add debug prints to help diagnose the issue
print(f"Current working directory: {os.getcwd()}")


import pandas as pd
import numpy as np

# Now import project modules
from src.utils.io_utils import getProcessedVideos, saveProcessedVideos
from src.processors.video_processor import videotokeypoints
from src.models.keypoints import get_keypoint_columns
from src.utils.keypoint_utils import normalize_keypoints
from src.main import process_all_videos

In [ ]:
# Add these to your imports
from src.config import PATH_CONFIG
from src.utils.notebook_utils import display_config_info, ensure_dir_exists

# Get paths from config
videos_in = PATH_CONFIG['videos_in']
data_out = PATH_CONFIG['data_out']

# Ensure output directory exists
if ensure_dir_exists(data_out):
    print(f"Created output directory: {data_out}")

# Display configuration information
display_config_info(videos_in, data_out, "Processing Configuration")


In [ ]:

metadata_file = "_LookitLaughter.test.xlsx"

#get metadata from excel file
metadata = pd.read_excel(os.path.join(videos_in, metadata_file))
metadata.head()

In [ ]:
processedvideos = getProcessedVideos(data_out)
processedvideos.head()

In [ ]:

# Option 1: Process videos using the refactored functions
forcemetadata = False
forceprocess = False 

# Process all videos - keep this line the same
process_all_videos(videos_in, data_out, metadata_file, forcemetadata, forceprocess)

In [ ]:
processedvideos = getProcessedVideos(data_out)
processedvideos.head()

## Create a set of normalised keypoint.csv. 

For modelling we want all movement data in standardised numerical format.
So 
1. Normalise x, y coordinates. Every coordinate is scaled to the range [0,1] where 1  is (framewidth, frameheight) respectively
2. We overwrite the 'person' column ["child","adult"] with numerical values [0,1] taken from 'index' column.

In [ ]:
# The normalization is now handled in the main process_all_videos function,
# but we can also do it separately for each video if needed:

processedvideos = getProcessedVideos(data_out)

for index, row in processedvideos.iterrows():
    if pd.isnull(row.get("Keypoints.normed")) or not os.path.exists(row.get("Keypoints.normed", "")):
        print(f"Normalizing keypoints for {row['VideoID']}")
        from src.main import normalize_and_save_keypoints
        normalize_and_save_keypoints(row.to_dict(), data_out)
    else:
        print(f"Already normalized {row['VideoID']}")

# Refresh the dataframe to see the updated values
processedvideos = getProcessedVideos(data_out)
processedvideos.head()